1. Load PDFs (Text, Image, Table) from Local Directory

In [1]:
import fitz
import os

pdf_dir = r"C:\Users\SHAHIL\Downloads\Agentic AI\18 may\agenticbatch2\TASK"
pdf_files = ["Global report on diabetes.pdf", "Understanding_Science.pdf"]

def extract_text_from_pdf(pdf_path):
    text = ""
    doc = fitz.open(pdf_path)
    for page in doc:
        text += page.get_text()
    return text

combined_text = ""
for file in pdf_files:
    file_path = os.path.join(pdf_dir, file)
    combined_text += extract_text_from_pdf(file_path)


2. Semantic Chunking

In [2]:
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdf_paths = [
    r"C:\Users\SHAHIL\Downloads\Agentic AI\18 may\agenticbatch2\TASK\Global report on diabetes.pdf",
    r"C:\Users\SHAHIL\Downloads\Agentic AI\18 may\agenticbatch2\TASK\Understanding_Science.pdf"
]

all_docs = []
for path in pdf_paths:
    loader = PyMuPDFLoader(path)
    docs = loader.load()
    all_docs.extend(docs)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(all_docs)


3. Embedding the Chunks

In [3]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

texts = [doc.page_content for doc in chunks]

chunk_embeddings = embedding_model.embed_documents(texts)


C:\Users\SHAHIL\AppData\Local\Temp\ipykernel_26828\1707740218.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
c:\Users\SHAHIL\Downloads\Agentic AI\18 may\agenticbatch2\TASK\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4. Store Inside Vector Database (ChromaDB)

In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

PERSIST_DIRECTORY = "./chroma_store"

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma(
    collection_name="pdf_chunks",
    embedding_function=embedding_model,
    persist_directory=PERSIST_DIRECTORY
)

vectorstore.add_texts(texts)

vectorstore.persist()

print("Chunks embedded and stored successfully in Chroma vectorstore.")


C:\Users\SHAHIL\AppData\Local\Temp\ipykernel_26828\928205960.py:8: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


Chunks embedded and stored successfully in Chroma vectorstore.


C:\Users\SHAHIL\AppData\Local\Temp\ipykernel_26828\928205960.py:16: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


5. Creating a index with all three index machnism(Flat, HNSW, IVF)

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.in_memory import InMemoryDocstore
import faiss
import numpy as np

# 1. Prepare text data
texts = [doc.page_content for doc in chunks]

# 2. Create embeddings
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents(texts)
embeddings = np.array(embeddings).astype("float32")
dimension = embeddings.shape[1]

# 3. Create FAISS indexes

# Flat index (exact search)
index_flat = faiss.IndexFlatL2(dimension)
index_flat.add(embeddings)

# HNSW index (approximate search)
index_hnsw = faiss.IndexHNSWFlat(dimension, 32)
index_hnsw.hnsw.efConstruction = 40
index_hnsw.add(embeddings)

# IVF index (inverted file index)
nlist = 100
quantizer = faiss.IndexFlatL2(dimension)
index_ivf = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
index_ivf.train(embeddings)
index_ivf.add(embeddings)
index_ivf.nprobe = 10


6. Creating Retrieval Pipeline

In [6]:
from typing import List, Tuple
from langchain.schema import Document

# Create mapping and docstore
docstore = InMemoryDocstore(dict(zip([str(i) for i in range(len(chunks))], chunks)))
index_to_docstore_id = {i: str(i) for i in range(len(chunks))}

# Wrap FAISS indexes into LangChain vectorstores
vectorstore_flat = FAISS(
    embedding_function=embedding_model.embed_query,
    index=index_flat,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

vectorstore_hnsw = FAISS(
    embedding_function=embedding_model.embed_query,
    index=index_hnsw,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

vectorstore_ivf = FAISS(
    embedding_function=embedding_model.embed_query,
    index=index_ivf,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# Direct FAISS search for Flat index
query = "What is diabetes?"
query_embedding = embedding_model.embed_query(query)
query_embedding = np.array(query_embedding).astype("float32").reshape(1, -1)

D, I = index_flat.search(query_embedding, 5)
print("Direct FAISS Search (Flat Index):")
print("Distances:", D)
print("Indices:", I)
for idx in I[0]:
    print(f"Doc {idx}: {texts[idx][:300]}...")


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


Direct FAISS Search (Flat Index):
Distances: [[0.55537176 0.71693385 0.7431896  0.7559942  0.79122806]]
Indices: [[ 27   0  28  92 127]]
Doc 27: 11
Background
KEY MESSAGES
Diabetes is a chronic, progressive disease characterized by elevated levels of 
blood glucose.
Diabetes of all types can lead to complications in many parts of the body and can 
increase the overall risk of dying prematurely.
Countries have committed to halt the rise in di...
Doc 0: GLOBAL REPORT 
ON DIABETES...
Doc 28: the insulin it produces (1). Raised 
blood glucose, a common effect of 
uncontrolled diabetes, may, over 
time, lead to serious damage to the 
heart, blood vessels, eyes, kidneys 
and nerves. More than 400 million 
people live with diabetes.
Type 1 diabetes (previously known 
as insulin-dependent, j...
Doc 92: PREVENTING DIABETES
PART 2...
Doc 127: 44
Preventing diabetes
REFERENCES
1.	 Definition, diagnosis and classification of diabetes mellitus and its complications. Part 1: Diagnosis and 
classific

7. Checking Retrieval Time

In [ ]:
import time

def measure_retriever_time(
    vectorstore, query: str, k: int = 5
) -> Tuple[List[Document], float]:
    start = time.time()
    docs = vectorstore.similarity_search(query, k=k)
    end = time.time()
    return docs, end - start

# Query and measure
docs_flat, time_flat = measure_retriever_time(vectorstore_flat, query)
docs_hnsw, time_hnsw = measure_retriever_time(vectorstore_hnsw, query)
docs_ivf, time_ivf = measure_retriever_time(vectorstore_ivf, query)

# Print retrieval times
print(f"\nFlat index retrieval time: {time_flat:.4f} sec")
print(f"HNSW index retrieval time: {time_hnsw:.4f} sec")
print(f"IVF index retrieval time: {time_ivf:.4f} sec")

# Print top results for inspection
def print_doc_previews(docs, label):
    print(f"\nTop {len(docs)} documents from {label} index:")
    for i, doc in enumerate(docs):
        preview = doc.page_content[:300].replace('\n', ' ')
        if len(doc.page_content) > 300:
            preview += "..."
        print(f"Doc {i+1} preview: {preview}\n")

print_doc_previews(docs_flat, "Flat")
print_doc_previews(docs_hnsw, "HNSW")
print_doc_previews(docs_ivf, "IVF")



Flat index retrieval time: 0.0084 sec
HNSW index retrieval time: 0.0079 sec
IVF index retrieval time: 0.0072 sec

Top 5 documents from Flat index:
Doc 1 preview: 11 Background KEY MESSAGES Diabetes is a chronic, progressive disease characterized by elevated levels of  blood glucose. Diabetes of all types can lead to complications in many parts of the body and can  increase the overall risk of dying prematurely. Countries have committed to halt the rise in di...

Doc 2 preview: GLOBAL REPORT  ON DIABETES

Doc 3 preview: the insulin it produces (1). Raised  blood glucose, a common effect of  uncontrolled diabetes, may, over  time, lead to serious damage to the  heart, blood vessels, eyes, kidneys  and nerves. More than 400 million  people live with diabetes. Type 1 diabetes (previously known  as insulin-dependent, j...

Doc 4 preview: PREVENTING DIABETES PART 2

Doc 5 preview: 44 Preventing diabetes REFERENCES 1.	 Definition, diagnosis and classification of diabetes mellitus and its com

8. Print Accuracy Score of Similarity Search

In [8]:
from typing import List, Tuple
import time
from langchain.schema import Document

def measure_retriever_with_scores(
    vectorstore, query: str, k: int = 5
) -> Tuple[List[Document], List[float], float]:
    """Retrieve documents with similarity scores and measure time"""
    start = time.time()
    docs_and_scores = vectorstore.similarity_search_with_score(query, k=k)
    docs, scores = zip(*docs_and_scores) if docs_and_scores else ([], [])
    end = time.time()
    return list(docs), list(scores), end - start

def print_retrieval_results(docs: List[Document], scores: List[float], method_name: str):
    """Print retrieval results with similarity scores"""
    print(f"\n{'-'*50}")
    print(f"Top {len(docs)} docs from {method_name} (with scores):")
    for i, (doc, score) in enumerate(zip(docs, scores)):
        print(f"\n#{i+1} [Score: {score:.4f}]")
        # Clean and truncate the text for display
        content = ' '.join(doc.page_content.split()[:30])  # Show first 30 words
        print(f"Content: {content}...")
        print(f"Metadata: {doc.metadata}") if hasattr(doc, 'metadata') else None

# Example usage:
query = "What is diabetes?"

# Get results with scores - using the correct function name now
docs_flat, scores_flat, time_flat = measure_retriever_with_scores(vectorstore_flat, query)
docs_hnsw, scores_hnsw, time_hnsw = measure_retriever_with_scores(vectorstore_hnsw, query)
docs_ivf, scores_ivf, time_ivf = measure_retriever_with_scores(vectorstore_ivf, query)

# Print results
print(f"\nQuery: '{query}'")
print(f"\nFlat index retrieval time: {time_flat:.4f} sec")
print_retrieval_results(docs_flat, scores_flat, "Flat Index")

print(f"\nHNSW index retrieval time: {time_hnsw:.4f} sec")
print_retrieval_results(docs_hnsw, scores_hnsw, "HNSW Index")

print(f"\nIVF index retrieval time: {time_ivf:.4f} sec")
print_retrieval_results(docs_ivf, scores_ivf, "IVF Index")


Query: 'What is diabetes?'

Flat index retrieval time: 0.0280 sec

--------------------------------------------------
Top 5 docs from Flat Index (with scores):

#1 [Score: 0.5554]
Content: 11 Background KEY MESSAGES Diabetes is a chronic, progressive disease characterized by elevated levels of blood glucose. Diabetes of all types can lead to complications in many parts of the...
Metadata: {'producer': 'Adobe PDF Library 15.0', 'creator': 'Adobe InDesign CC 2015 (Macintosh)', 'creationdate': '2016-04-01T14:07:06+02:00', 'source': 'C:\\Users\\SHAHIL\\Downloads\\Agentic AI\\18 may\\agenticbatch2\\TASK\\Global report on diabetes.pdf', 'file_path': 'C:\\Users\\SHAHIL\\Downloads\\Agentic AI\\18 may\\agenticbatch2\\TASK\\Global report on diabetes.pdf', 'total_pages': 88, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2016-04-01T14:07:57+02:00', 'trapped': '', 'modDate': "D:20160401140757+02'00'", 'creationDate': "D:20160401140706+02'00'", 'page': 1

9. Performing the reranking using BM25

In [9]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

# Preprocess text (tokenize, remove stopwords/punctuation)
def preprocess(text):
    tokens = text.lower().split()
    tokens = [t for t in tokens if t not in _stop_words.ENGLISH_STOP_WORDS and t not in string.punctuation]
    return tokens

# Get all document texts
doc_texts = [doc.page_content for doc in chunks]
tokenized_corpus = [preprocess(doc) for doc in doc_texts]

# Initialize BM25
bm25 = BM25Okapi(tokenized_corpus)

# Function to rerank FAISS results with BM25
def rerank_with_bm25(query, faiss_docs, bm25_model, top_k=5):
    query_tokens = preprocess(query)
    doc_indices = [int(doc.metadata.get("id", i)) for i, doc in enumerate(faiss_docs)]
    bm25_scores = bm25_model.get_scores(query_tokens)
    
    # Get BM25 scores for the retrieved docs
    scored_docs = [(doc, bm25_scores[i]) for i, doc in zip(doc_indices, faiss_docs)]
    scored_docs.sort(key=lambda x: x[1], reverse=True)  # Sort by BM25 score
    
    return [doc for doc, score in scored_docs[:top_k]]

# Example usage
bm25_reranked_flat = rerank_with_bm25(query, docs_flat, bm25)
bm25_reranked_hnsw = rerank_with_bm25(query, docs_hnsw, bm25)
bm25_reranked_ivf = rerank_with_bm25(query, docs_ivf, bm25)

# Print BM25 results
print("\nBM25 Re-ranked Results (Flat Index):")
for i, doc in enumerate(bm25_reranked_flat):
    print(f"\n#{i+1}: {' '.join(doc.page_content.split()[:30])}...")


BM25 Re-ranked Results (Flat Index):

#1: 11 Background KEY MESSAGES Diabetes is a chronic, progressive disease characterized by elevated levels of blood glucose. Diabetes of all types can lead to complications in many parts of the...

#2: GLOBAL REPORT ON DIABETES...

#3: the insulin it produces (1). Raised blood glucose, a common effect of uncontrolled diabetes, may, over time, lead to serious damage to the heart, blood vessels, eyes, kidneys and nerves....

#4: PREVENTING DIABETES PART 2...

#5: 44 Preventing diabetes REFERENCES 1. Definition, diagnosis and classification of diabetes mellitus and its complications. Part 1: Diagnosis and classification of diabetes mellitus. WHO/NCD/NCS/99.2. Geneva: World Health Organization; 1999. 2....


10: Write a Prompt Template for LLM

In [ ]:
from langchain.prompts import PromptTemplate

# Defining the prompt template
prompt_template = PromptTemplate(
    input_variables=["query", "documents"],
    template="""
    You are an expert AI assistant. Answer the following question based on the provided context.

    Question: {query}

    Context:
    {documents}

    Instructions:
    1. Provide a clear, concise answer.
    2. If the context doesn't contain the answer, say "I could not find an answer."
    3. Cite sources if available (e.g., "According to Document 1...").

    Answer:
    """
)

# Example: Format the prompt with retrieved docs
documents_str = "\n\n".join([f"Document {i+1}:\n{doc.page_content[:500]}..." for i, doc in enumerate(bm25_reranked_flat)])

prompt = prompt_template.format(
    query=query,
    documents=documents_str
)

print("===== GENERATED PROMPT =====")
print(prompt)

===== GENERATED PROMPT =====

    You are an expert AI assistant. Answer the following question based on the provided context.

    Question: What is diabetes?

    Context:
    Document 1:
11
Background
KEY MESSAGES
Diabetes is a chronic, progressive disease characterized by elevated levels of 
blood glucose.
Diabetes of all types can lead to complications in many parts of the body and can 
increase the overall risk of dying prematurely.
Countries have committed to halt the rise in diabetes, to reduce diabetes-related 
premature mortality and to improve access to essential diabetes medicines and 
basic technologies.
Effective tools are available to prevent type 2 diabetes and to i...

Document 2:
GLOBAL REPORT 
ON DIABETES...

Document 3:
the insulin it produces (1). Raised 
blood glucose, a common effect of 
uncontrolled diabetes, may, over 
time, lead to serious damage to the 
heart, blood vessels, eyes, kidneys 
and nerves. More than 400 million 
people live with diabetes.
Type 1 d

11. Generting a oputput through LLM and 12. Rendering Output into a DOCX file

In [11]:
import os
from docx import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

queries = [
    "What is diabetes?",
    "What are the symptoms of hypertension?"
]
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
FILENAME = "medical_report.docx"

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.5
)

def generate_llm_response(query):
    prompt = f"""
    You are a medical expert. Answer this question concisely:

    Question: {query}

    Instructions:
    1. Provide a 2-3 sentence answer.
    2. If unsure, say "I couldn't find a definitive answer."
    """
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error generating response: {e}. Check your GOOGLE_API_KEY or quota."

# Function to render output to DOCX (append if file exists)
def render_to_docx(query, response_text, filename=FILENAME):
    try:
        # Check if file exists
        if os.path.exists(filename):
            doc = Document(filename)
        else:
            doc = Document()
            doc.add_heading("Medical Report", level=1)
        
        # Add question and answer
        doc.add_paragraph(f"Question: {query}")
        doc.add_paragraph(f"Answer: {response_text}")
        doc.add_paragraph("")
        doc.save(filename)
        print(f"✅ Report updated in '{filename}'")
    except Exception as e:
        print(f"Error saving DOCX: {e}")

# Process multiple questions and render to DOCX
for query in queries:
    print(f"\nProcessing query: {query}")
    response_text = generate_llm_response(query)
    print("===== GEMINI RESPONSE =====")
    print(response_text)
    render_to_docx(query, response_text)


Processing query: What is diabetes?
===== GEMINI RESPONSE =====
Diabetes is a group of metabolic disorders characterized by persistently high levels of blood glucose.  This hyperglycemia results from defects in insulin secretion, insulin action, or both.  The high blood sugar can damage nerves, blood vessels, and organs over time.
✅ Report updated in 'medical_report.docx'

Processing query: What are the symptoms of hypertension?
===== GEMINI RESPONSE =====
Hypertension often has no noticeable symptoms.  However,  severe, untreated hypertension can lead to headaches, dizziness, shortness of breath, and nosebleeds.
✅ Report updated in 'medical_report.docx'


This section processes additional medical questions and appends the responses to the existing `medical_report.docx` file. The questions focus on common medical topics, and the answers are generated concisely (2-3 sentences) as per the prompt instructions. The code ensures that new question-answer pairs are appended without overwriting the existing content in the DOCX file.

In [12]:
queries = [
    "What are the common causes of migraines?",
    "How is asthma treated?",
    "What are the risk factors for heart disease?",
    "What is hypothyroidism and its symptoms?",
    "How can type 2 diabetes be managed?",
    "What are the signs of an allergic reaction?",
    "What is the purpose of a colonoscopy?",
    "How does high cholesterol affect the body?",
    "What are the symptoms of a stroke?",
    "What lifestyle changes prevent osteoporosis?"
]
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
FILENAME = "medical_report.docx" 


llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.5
)

def generate_llm_response(query):
    prompt = f"""
    You are a medical expert. Answer this question concisely:

    Question: {query}

    Instructions:
    1. Provide a 2-3 sentence answer.
    2. If unsure, say "I couldn't find a definitive answer."
    """
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error generating response: {e}. Check your GOOGLE_API_KEY or quota."

# Function to render output to DOCX (append if file exists)
def render_to_docx(query, response_text, filename=FILENAME):
    try:
        # Check if file exists
        if os.path.exists(filename):
            doc = Document(filename)
        else:
            doc = Document()
            doc.add_heading("Medical Report", level=1)
        
        # Add question and answer
        doc.add_paragraph(f"Question: {query}")
        doc.add_paragraph(f"Answer: {response_text}")
        doc.add_paragraph("")
        doc.save(filename)
        print(f"✅ Report updated in '{filename}'")
    except Exception as e:
        print(f"Error saving DOCX: {e}")

# Process new questions and append to DOCX
for query in queries:
    print(f"\nProcessing query: {query}")
    response_text = generate_llm_response(query)
    print("===== GEMINI RESPONSE =====")
    print(response_text)
    render_to_docx(query, response_text)


Processing query: What are the common causes of migraines?
===== GEMINI RESPONSE =====
Common migraine triggers include hormonal changes, stress, certain foods or drinks, lack of sleep, and changes in weather patterns.  Underlying genetic predisposition also plays a significant role.
✅ Report updated in 'medical_report.docx'

Processing query: How is asthma treated?
===== GEMINI RESPONSE =====
Asthma treatment involves quick-relief medications like bronchodilators to alleviate symptoms during attacks, and long-term control medications, such as inhaled corticosteroids, to reduce inflammation and prevent attacks.  Lifestyle modifications, like avoiding triggers, are also crucial.
✅ Report updated in 'medical_report.docx'

Processing query: What are the risk factors for heart disease?
===== GEMINI RESPONSE =====
Major risk factors for heart disease include high blood pressure, high cholesterol, smoking, diabetes, obesity, physical inactivity, unhealthy diet, and family history of heart d

This section handles 10 questions from the Understanding_Science.pdf file. The questions cover topics like properties of matter, forces, motion, fluids, and heat. Each answer is short and clear (2–3 sentences). The answers are saved in a file called science_report.docx. If the file doesn’t exist, it will be created. If it already exists, the new questions and answers will be added to it. This will not change or affect the medical_report.docx file.

In [14]:
import os
from docx import Document
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

queries = [
    "What are the three states of matter and their key properties?",
    "How does Boyle’s Law explain the behavior of gases?",
    "What is the role of surface tension in capillary action?",
    "How does gravity affect the motion of objects?",
    "What is the difference between weight and mass in physics?",
    "How do levers work to amplify force?",
    "What is the significance of Newton’s First Law of Motion?",
    "How does thermal expansion affect solids and liquids?",
    "What is Archimedes’ Principle and its application in hydrostatics?",
    "How do convection currents transfer heat in fluids?"
]
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
FILENAME = "science_report.docx"

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.5
)

def generate_llm_response(query):
    prompt = f"""
    You are a science expert. Answer this question concisely:

    Question: {query}

    Instructions:
    1. Provide a 2-3 sentence answer.
    2. If unsure, say "I couldn't find a definitive answer."
    """
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        return f"Error generating response: {e}. Check your GOOGLE_API_KEY or quota."

def render_to_docx(query, response_text, filename=FILENAME):
    try:
        # Check if file exists
        if os.path.exists(filename):
            doc = Document(filename)
        else:
            doc = Document()
            doc.add_heading("Science Report", level=1)
        
        # Add question and answer
        doc.add_paragraph(f"Question: {query}")
        doc.add_paragraph(f"Answer: {response_text}")
        doc.add_paragraph("")
        doc.save(filename)
        print(f"✅ Report updated in '{filename}'")
    except Exception as e:
        print(f"Error saving DOCX: {e}")

# Process questions and save to DOCX
for query in queries:
    print(f"\nProcessing query: {query}")
    response_text = generate_llm_response(query)
    print("===== GEMINI RESPONSE =====")
    print(response_text)
    render_to_docx(query, response_text)


Processing query: What are the three states of matter and their key properties?


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 15
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
].


===== GEMINI RESPONSE =====
The three states of matter are solid (definite shape and volume), liquid (definite volume, indefinite shape), and gas (indefinite shape and volume).  These states are determined by the arrangement and energy of the constituent particles.
✅ Report updated in 'science_report.docx'

Processing query: How does Boyle’s Law explain the behavior of gases?
===== GEMINI RESPONSE =====
Boyle's Law states that at constant temperature, the volume of a gas is inversely proportional to its pressure.  This means that as pressure increases, volume decreases, and vice versa,  demonstrating the relationship between a gas's pressure and volume.
✅ Report updated in 'science_report.docx'

Processing query: What is the role of surface tension in capillary action?
===== GEMINI RESPONSE =====
Surface tension, the cohesive force between liquid molecules, creates an inward pull at the liquid's surface.  This pull, coupled with adhesive forces between the liquid and the tube's walls, 